<a href="https://colab.research.google.com/github/yoonni0908/computer-vision-study/blob/main/VGGnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn

In [2]:
cfgs = { "A": [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
         "B": [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
         "D": [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"],
         "E": [64, 64, "M", 128, 128, "M", 256, 256, 256, 256, "M", 512, 512, 512, 512, "M", 512, 512, 512, 512, "M"] }

In [9]:
class VGG(nn.Module):
    def __init__(self, cfg, batch_norm, num_classes = 1000, init_weights=True, drop_p = 0.5):
        super().__init__()

        self.features = self.make_layers(cfg)
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7)) # 7x7이 되도록 avg pooling 하는 녀석
        self.classifier = nn.Sequential(nn.Linear(512 * 7 * 7, 4096),
                                        nn.ReLU(),
                                        nn.Dropout(p=drop_p),
                                        nn.Linear(4096, 4096),
                                        nn.ReLU(),
                                        nn.Dropout(p=drop_p),
                                        nn.Linear(4096, num_classes))

        if init_weights:
            for m in self.modules():
                if isinstance(m, nn.Conv2d):
                    nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu") # 합성곱 레이어의 가중치를 Kaiming Normal 초기화 방법
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0) # bias 초기화
                elif isinstance(m, nn.Linear):
                    nn.init.normal_(m.weight, 0, 0.01) #선형 레이어의 weight를 normal distribution에서 무작위로 초기화, 0은 평균, 0.01은 표준편차
                    nn.init.constant_(m.bias, 0) # bias 초기화

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def make_layers(self, cfg, batch_norm = False):
        layers = []
        in_channels = 3
        for v in cfg:
            if type(v) == int: # in면 conv
                if batch_norm:
                    layers += [nn.Conv2d(in_channels, v, 3, padding=1), # list append
                              nn.BatchNorm2d(v),
                              nn.ReLU()]
                else:
                    layers += [nn.Conv2d(in_channels, v, 3, padding=1),
                             nn.ReLU()]
                in_channels = v
            else:
                layers += [nn.MaxPool2d(2)] # string이면 pooling
        return nn.Sequential(*layers)

In [10]:
avgpool = nn.AdaptiveAvgPool2d((4, 4))
print(avgpool(torch.randn(2,3,32,32)).shape)
print(avgpool(torch.randn(2,3,2,2))) # 작은 놈이 들어오면 늘려서라도 맞춰준다
# print(avgpool(torch.randn(2,3,1,1))) # 값을 복제 시켜놓음

torch.Size([2, 3, 4, 4])
tensor([[[[-0.2651, -0.2651,  1.2632,  1.2632],
          [-0.2651, -0.2651,  1.2632,  1.2632],
          [ 0.0675,  0.0675,  0.6010,  0.6010],
          [ 0.0675,  0.0675,  0.6010,  0.6010]],

         [[-1.6959, -1.6959, -0.7545, -0.7545],
          [-1.6959, -1.6959, -0.7545, -0.7545],
          [ 0.2379,  0.2379, -1.2204, -1.2204],
          [ 0.2379,  0.2379, -1.2204, -1.2204]],

         [[-1.3491, -1.3491,  0.6328,  0.6328],
          [-1.3491, -1.3491,  0.6328,  0.6328],
          [ 0.1624,  0.1624,  0.9800,  0.9800],
          [ 0.1624,  0.1624,  0.9800,  0.9800]]],


        [[[ 0.2197,  0.2197,  2.0498,  2.0498],
          [ 0.2197,  0.2197,  2.0498,  2.0498],
          [-0.0237, -0.0237,  1.3253,  1.3253],
          [-0.0237, -0.0237,  1.3253,  1.3253]],

         [[-0.0531, -0.0531, -0.6551, -0.6551],
          [-0.0531, -0.0531, -0.6551, -0.6551],
          [-1.3351, -1.3351, -0.5670, -0.5670],
          [-1.3351, -1.3351, -0.5670, -0.5670]],

   

In [11]:
model = VGG(cfgs["E"], batch_norm=False)
# print(model)
!pip install torchinfo
from torchinfo import summary
summary(model, input_size=(2,3,224,224), device='cpu')

Layer (type:depth-idx)                   Output Shape              Param #
VGG                                      [2, 1000]                 --
├─Sequential: 1-1                        [2, 512, 7, 7]            --
│    └─Conv2d: 2-1                       [2, 64, 224, 224]         1,792
│    └─ReLU: 2-2                         [2, 64, 224, 224]         --
│    └─Conv2d: 2-3                       [2, 64, 224, 224]         36,928
│    └─ReLU: 2-4                         [2, 64, 224, 224]         --
│    └─MaxPool2d: 2-5                    [2, 64, 112, 112]         --
│    └─Conv2d: 2-6                       [2, 128, 112, 112]        73,856
│    └─ReLU: 2-7                         [2, 128, 112, 112]        --
│    └─Conv2d: 2-8                       [2, 128, 112, 112]        147,584
│    └─ReLU: 2-9                         [2, 128, 112, 112]        --
│    └─MaxPool2d: 2-10                   [2, 128, 56, 56]          --
│    └─Conv2d: 2-11                      [2, 256, 56, 56]          29

In [12]:
x = torch.randn(2,3,224,224)
print(model(x).shape)

torch.Size([2, 1000])
